In [1]:
import mne
import os
import numpy as np
import matplotlib.pyplot as plt


In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import networkx as nx

In [3]:
print('TensorFlow version: {}'.format(tf.__version__))
print('Eager execution: {}'.format(tf.executing_eagerly()))

TensorFlow version: 2.2.0
Eager execution: True


In [4]:
sbj='005'
rec= [3,7]
categories = ['T1','T2']
train_data = []
for rec in rec:
    edf_loc= '/media/biraj/DATA/Motor Imagery Dataset/S'+sbj+'/S'+sbj+'R0'+str(rec)+'.edf'
    Data = mne.io.read_raw_edf(edf_loc)
    events, event_id=mne.events_from_annotations(Data)
    epochs=mne.Epochs(Data, events, event_id, tmin=0, tmax=4, baseline=(None,None))
    
    montage = mne.channels.make_standard_montage('standard_1020')
    Data.rename_channels(lambda x: x.strip('.'))
    ch_map = {'Fc5': 'FC5',
          'Fc3': 'FC3',
          'Fc1': 'FC1',
          'Fcz': 'FCz',
          'Fc2': 'FC2',
          'Fc4': 'FC4',
          'Fc6': 'FC6',
          'Cp5': 'CP5',
          'Cp3': 'CP3',
          'Cp1': 'CP1',
          'Cpz': 'CPz',
          'Cp2': 'CP2',
          'Cp4': 'CP4',
          'Cp6': 'CP6',
          'Af7': 'AF7',
          'Af3': 'AF3',
          'Afz': 'AFz',
          'Af4': 'AF4',
          'Af8': 'AF8',
          'Ft7': 'FT7',
          'Ft8': 'FT8',
          'Tp7': 'TP7',
          'Tp8': 'TP8',
          'Po7': 'PO7',
          'Po3': 'PO3',
          'Poz': 'POz',
          'Po4': 'PO4',
          'Po8': 'PO8'}
    Data.rename_channels(ch_map)
    Data.set_montage(montage)
    include ={'Fp1', 'Fp2','AF3','AF4','AF7','AF8',
          'FC1','FC2','FC3','FC4','FC5','FC6',
          'C1','C2','C3','C4','C5','C6',
          'CP1','CP2','CP3','CP4','CP5','CP6'}
    picks=mne.pick_channels(Data.ch_names, include, ordered=True)
    events, event_id=mne.events_from_annotations(Data)
    epochs=mne.Epochs(Data, events, event_id=event_id, tmin=0, tmax=4, picks= picks, baseline=(None,None))
    
    T1=epochs['T1'].get_data()
    T2=epochs['T2'].get_data()
    
    info = Data.info
    fmin=4
    fmax=12
    how_many=10
    
    
    for i in range(3):
        if i==0:
            continue
        
        T=epochs['T'+str(i)]
        for j in range(7):
            plv, freqs, times, n_epochs, n_tapers=mne.connectivity.spectral_connectivity(T[j],method='plv', indices=None, mode='cwt_morlet', sfreq=info['sfreq'], fmin=fmin, fmax=fmax, faverage=True, cwt_freqs=np.linspace(fmin,fmax,how_many), cwt_n_cycles=1, n_jobs=1)
            G=nx.Graph()
            centrality=[]
            for t in range(len(plv[1,1,0,:])):
                min_plv=0
                max_plv=1
                adj=plv[:,:,0,t]
                G.clear()
                G.add_nodes_from(epochs.ch_names)
                pos=nx.circular_layout(G)
                for k in range(len(epochs.ch_names)-1):
                    for l in range(k):
                        #if (adj[k,l]<=min_plv or adj[k,l]>=max_plv):
                            #continue
                        w_edges= [(epochs.ch_names[k],epochs.ch_names[l],adj[k,l])]
                        G.add_weighted_edges_from(w_edges)

                H=nx.minimum_spanning_tree(G, weight=1)
                centrality.append(nx.eigenvector_centrality(H).values())
                
            lst=[]
            for t in centrality:
                lst.append(list(t))
                
            train_data.append([lst, i])


    
    

Extracting EDF parameters from /media/biraj/DATA/Motor Imagery Dataset/S005/S005R03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['T0', 'T1', 'T2']
30 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Used Annotations descriptions: ['T0', 'T1', 'T2']
30 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 7 events and 641 original time points ...
0 bad epochs dropped
Loading data for 8 events and 641 original time points ...
0 bad epochs dropped
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 276 connections
    using t=0.000s..4.000s for estimation (641 points)
    frequencies: 4.0Hz..12.0Hz (10 points)
    connectivity scores will be averaged for each band
    using CWT with Morlet wavelets to estimate spectra
  

EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['T0', 'T1', 'T2']
30 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Used Annotations descriptions: ['T0', 'T1', 'T2']
30 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 8 events and 641 original time points ...
0 bad epochs dropped
Loading data for 7 events and 641 original time points ...
0 bad epochs dropped
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 276 connections
    using t=0.000s..4.000s for estimation (641 points)
    frequencies: 4.0Hz..12.0Hz (10 points)
    connectivity scores will be averaged for each band
    using CWT with Morlet wavelets to estimate spectra
    the following metrics will be computed: PLV
    computing connectivity for epoch 1
    as

In [5]:
x = []
y = []

In [6]:
for features, label in train_data:
    x.append(features)
    y.append(label)
    

In [7]:
x=np.array(np.transpose(x,(0,2,1)))


In [8]:
x.shape

(28, 24, 641)

In [9]:
x=np.reshape(x,(-1,24,641,1))
y=np.array(y)

In [10]:
print(x.shape)
print(len(y))
print(x.shape[1:])

(28, 24, 641, 1)
28
(24, 641, 1)


In [11]:
model = Sequential()
model.add(Conv2D(64, (3,3), input_shape = x.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

model.fit(x, y, batch_size = 32, epochs=3, validation_split=0.1)



Epoch 1/3
1/1 [==============================] - 1s 1s/step - loss: 0.6939 - accuracy: 0.0000e+00 - val_loss: -2.7739 - val_accuracy: 0.0000e+00
Epoch 2/3
1/1 [==============================] - 0s 213ms/step - loss: -1.1885 - accuracy: 0.5600 - val_loss: -7.4812 - val_accuracy: 0.0000e+00
Epoch 3/3
1/1 [==============================] - 0s 171ms/step - loss: -3.2914 - accuracy: 0.5600 - val_loss: -14.8355 - val_accuracy: 0.0000e+00
